In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 17:19:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/30 17:19:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [4]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
change_colum = data.withColumnRenamed('_c0', 'lable').withColumnRenamed('_c1', 'text')

In [8]:
change_colum.show()

+-----+--------------------+
|lable|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [15]:
change_colum = change_colum.withColumn('length_text', length(change_colum['text']))

In [16]:
change_colum.show()

+-----+--------------------+-----------+
|lable|                text|length_text|
+-----+--------------------+-----------+
|  ham|Go until jurong p...|        111|
|  ham|Ok lar... Joking ...|         29|
| spam|Free entry in 2 a...|        155|
|  ham|U dun say so earl...|         49|
|  ham|Nah I don't think...|         61|
| spam|FreeMsg Hey there...|        147|
|  ham|Even my brother i...|         77|
|  ham|As per your reque...|        160|
| spam|WINNER!! As a val...|        157|
| spam|Had your mobile 1...|        154|
|  ham|I'm gonna be home...|        109|
| spam|SIX chances to wi...|        136|
| spam|URGENT! You have ...|        155|
|  ham|I've been searchi...|        196|
|  ham|I HAVE A DATE ON ...|         35|
| spam|XXXMobileMovieClu...|        149|
|  ham|Oh k...i'm watchi...|         26|
|  ham|Eh u remember how...|         81|
|  ham|Fine if thats th...|         56|
| spam|England v Macedon...|        155|
+-----+--------------------+-----------+
only showing top

In [21]:
change_colum.groupby('lable').avg('length_text').show()

+-----+-----------------+
|lable| avg(length_text)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [25]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF ,VectorAssembler, StringIndexer

In [26]:
token = Tokenizer(inputCol='text', outputCol='token_text')
stop_word = StopWordsRemover(inputCol='token_text', outputCol='stop_text')
count_word = CountVectorizer(inputCol='stop_text', outputCol = 'count_text')
idf = IDF(inputCol='count_text', outputCol = 'hash_text')
label_numeric = StringIndexer(inputCol='lable', outputCol='label')

In [27]:
assembler = VectorAssembler(inputCols=['hash_text','label'], outputCol='features')

In [28]:
from pyspark.ml.classification import NaiveBayes

In [29]:
nb = NaiveBayes()

In [30]:
from pyspark.ml.pipeline import Pipeline

In [32]:
data_pipeline = Pipeline(stages=[token, stop_word, count_word, idf, label_numeric, assembler])

In [33]:
cleaner = data_pipeline.fit(change_colum)

In [51]:
clean_data = cleaner.transform(change_colum)

In [52]:
clean_data = clean_data.select('label', 'features')

In [53]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [54]:
train,test = clean_data.randomSplit([0.7,0.3])

In [55]:
spam_dector = nb.fit(train)

23/12/30 17:56:31 WARN DAGScheduler: Broadcasting large task binary with size 1207.8 KiB
23/12/30 17:56:31 WARN DAGScheduler: Broadcasting large task binary with size 1185.0 KiB


In [58]:
change_colum.printSchema()

root
 |-- lable: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length_text: integer (nullable = true)



In [59]:
test_result = spam_dector.transform(test)

In [60]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-495.94390768703...|[1.0,8.0042945731...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-694.68805504212...|[1.0,3.8760419852...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-993.39402733565...|[1.0,1.6458198485...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-533.75871788095...|[1.0,6.4739991844...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-422.01690880519...|[1.0,4.4299205068...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1151.7379564992...|[1.0,1.6776800939...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-558.39583926003...|[1.0,4.0803259951...|       0.0|
|  0.0|(13424,[0,1,16,19...|[-689.21617828353...|[1.0,1.1943449004...|       0.0|
|  0.0|(13424,[0,1,71,10...|[-585.71551442198...|[1.0,4.6378872723...|       0.0|
|  0.0|(13424,[0

23/12/30 18:02:13 WARN DAGScheduler: Broadcasting large task binary with size 1412.0 KiB
23/12/30 18:02:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/30 18:02:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [61]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [62]:
eval_obj = MulticlassClassificationEvaluator()

In [63]:
eval_model = eval_obj.evaluate(test_result)

23/12/30 18:04:38 WARN DAGScheduler: Broadcasting large task binary with size 1417.0 KiB


In [64]:
eval_model

0.9180605493793972